# K-mer 
Basic K-mer counting.

In [1]:
import time
t = time.time()
time.strftime('%Y-%m-%d %H:%M:%S %Z', time.localtime(t))

'2021-07-06 14:16:13 EDT'

In [2]:
PC_SEQUENCES=32000
NC_SEQUENCES=32000
RNA_LEN=32
CDS_LEN=16

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
import sys
IN_COLAB = False
try:
    from google.colab import drive
    IN_COLAB = True
except:
    pass
if IN_COLAB:
    print("On Google CoLab, mount cloud-local file, get our code from GitHub.")
    PATH='/content/drive/'
    #drive.mount(PATH,force_remount=True)  # hardly ever need this
    #drive.mount(PATH)    # Google will require login credentials
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
    import requests
    r = requests.get('https://raw.githubusercontent.com/ShepherdCode/Soars2021/master/SimTools/RNA_describe.py')
    with open('RNA_describe.py', 'w') as f:
        f.write(r.text)  
    from RNA_describe import ORF_counter
    from RNA_describe import Random_Base_Oracle
else:
        print("CoLab not working. On my PC, use relative paths.")
        DATAPATH='data/'  # must end in "/"
        sys.path.append("..") # append parent dir in order to use sibling dirs
        from SimTools.RNA_describe import ORF_counter,Random_Base_Oracle
MODELPATH="BestModel"  # saved on cloud instance and lost after logout
#MODELPATH=DATAPATH+MODELPATH  # saved on Google Drive but requires login

CoLab not working. On my PC, use relative paths.


## Sample data

In [5]:
rbo=Random_Base_Oracle(RNA_LEN,True)
pc_all,nc_all = rbo.get_partitioned_sequences(CDS_LEN,10) # just testing
pc_all,nc_all = rbo.get_partitioned_sequences(CDS_LEN,PC_SEQUENCES)
print("Use",len(pc_all),"PC seqs")
print("Use",len(nc_all),"NC seqs")

It took 44 trials to reach 10 per class.
It took 139005 trials to reach 32000 per class.
Use 32000 PC seqs
Use 32000 NC seqs


## Primitive K-mer counting

In [8]:
def make_kmer_keys(K):
    shorter_kmers=['']
    for i in range(K):
        longer_kmers=[]
        for mer in shorter_kmers:
            # No support for N or any non-ACGT bases.
            longer_kmers.append(mer+'A')
            longer_kmers.append(mer+'C')
            longer_kmers.append(mer+'G')
            longer_kmers.append(mer+'T')
        shorter_kmers = longer_kmers
    return shorter_kmers
def make_kmer_dict(keys,init=0):
    return dict.fromkeys(keys,init)
def make_dict_123():
    keys=make_kmer_keys(1)
    keys.extend(make_kmer_keys(2))
    keys.extend(make_kmer_keys(3))
    counts = make_kmer_dict(keys)
    return counts
def update_Kmer_count(counts,K,rna):
    L = len(rna)
    padding=" "*(K-1)
    padded=rna+padding
    for i in range(0,L-K+1):
        kmer=padded[i:i+K]
        counts[kmer] += 1
def update_dict_123(counts,sample):
    update_Kmer_count(counts,1,sample)
    update_Kmer_count(counts,2,sample)
    update_Kmer_count(counts,3,sample)
    return counts

In [9]:
all_counts = make_dict_123()
sample = "ACCGGGTTTTACGTACGT"
all_counts = update_dict_123(all_counts,sample)
print("Updated counts:",all_counts)

Updated counts: {'A': 3, 'C': 4, 'G': 5, 'T': 6, 'AA': 0, 'AC': 3, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 1, 'CG': 3, 'CT': 0, 'GA': 0, 'GC': 0, 'GG': 2, 'GT': 3, 'TA': 2, 'TC': 0, 'TG': 0, 'TT': 3, 'AAA': 0, 'AAC': 0, 'AAG': 0, 'AAT': 0, 'ACA': 0, 'ACC': 1, 'ACG': 2, 'ACT': 0, 'AGA': 0, 'AGC': 0, 'AGG': 0, 'AGT': 0, 'ATA': 0, 'ATC': 0, 'ATG': 0, 'ATT': 0, 'CAA': 0, 'CAC': 0, 'CAG': 0, 'CAT': 0, 'CCA': 0, 'CCC': 0, 'CCG': 1, 'CCT': 0, 'CGA': 0, 'CGC': 0, 'CGG': 1, 'CGT': 2, 'CTA': 0, 'CTC': 0, 'CTG': 0, 'CTT': 0, 'GAA': 0, 'GAC': 0, 'GAG': 0, 'GAT': 0, 'GCA': 0, 'GCC': 0, 'GCG': 0, 'GCT': 0, 'GGA': 0, 'GGC': 0, 'GGG': 1, 'GGT': 1, 'GTA': 1, 'GTC': 0, 'GTG': 0, 'GTT': 1, 'TAA': 0, 'TAC': 2, 'TAG': 0, 'TAT': 0, 'TCA': 0, 'TCC': 0, 'TCG': 0, 'TCT': 0, 'TGA': 0, 'TGC': 0, 'TGG': 0, 'TGT': 0, 'TTA': 1, 'TTC': 0, 'TTG': 0, 'TTT': 2}
